Updated 26 Sept 2022
Press shift-enter to run each cell. You can edit the code in each cell, and you can also re-run each cell. The lines that start with # are comments that give you instructions on what you can edit.

In [ ]:
#this cell imports the functions needed in the notebook
from __future__ import print_function
import numpy as np
import os
import numbers

import ipywidgets as widgets

%matplotlib widget
import matplotlib.pyplot as plt
import matplotlib.animation as animation

from IPython.display import display

from reconstruction import recon_setup, recon
import dxchange
import h5py

import pickle
recons_todo = [] #list of recon_dictionary variables so that you can accumulate a bunch of them and then run them at once.

In [ ]:
inputSubFolderName = "ALS-12345_name" #this should be the name of the folder in the file browser to the left that has data
outputSubfolderName = "reconstructions" #this can be anything you want, I usually choose the current date

inputPath = os.path.join("/alsdata", inputSubFolderName)

if os.path.isdir("/alsuser/pscratch"):
    wheretosave = "pscratch"
elif os.path.isdir("/alsuser/cscratch"):  
    wheretosave = "cscratch"
else:
    wheretosave = "notebooks"
    
outputPath = os.path.join("/alsuser/", wheretosave, outputSubfolderName)    

if not os.path.exists(outputPath):
    os.mkdir(outputPath)
pickledparamsfile = f'{outputSubfolderName}.pkl'
filenamelist = os.listdir(inputPath)
filenamelist.sort()
for i in range(len(filenamelist)-1,np.maximum(len(filenamelist)-10000,-1),-1):
    print(f'{i}: {filenamelist[i]}')

In [ ]:
filename = filenamelist[3] #update this number with the index of the file you want to process from the directory listing generated in the previous cell
filetype = 'dxfile' #should be dxfile for data since mid 2021. For data before that, use 'als' instead

if (filetype == 'als') or (filetype == 'als1131'):
    datafile = h5py.File(os.path.join(inputPath,filename), 'r')
    gdata = dict(dxchange.reader._find_dataset_group(datafile).attrs)
    pxsize = float(gdata['pxsize'])
    numslices = int(gdata['nslices'])
    numangles = int(gdata['nangles'])
    angularrange = float(gdata['arange'])
    numrays = int(gdata['nrays'])
    inter_bright = int(gdata['i0cycle'])
    propagation_dist = float(gdata['Camera_Z_Support'])
    kev = float(gdata['Mono_Energy']) / 1000
elif (filetype == 'dxfile'):
    numslices = int(dxchange.read_hdf5(os.path.join(inputPath, filename), "/measurement/instrument/detector/dimension_y")[0])
    numrays = int(dxchange.read_hdf5(os.path.join(inputPath, filename), "/measurement/instrument/detector/dimension_x")[0])
    pxsize = dxchange.read_hdf5(os.path.join(inputPath, filename), "/measurement/instrument/detector/pixel_size")[0] / 10.0  # /10 to convert units from mm to cm
    numangles = int(dxchange.read_hdf5(os.path.join(inputPath, filename), "/process/acquisition/rotation/num_angles")[0])
    propagation_dist = dxchange.read_hdf5(os.path.join(inputPath, filename), "/measurement/instrument/camera_motor_stack/setup/camera_distance")[0]
    kev = dxchange.read_hdf5(os.path.join(inputPath, filename), "/measurement/instrument/monochromator/energy")[0] / 1000
    angularrange = dxchange.read_hdf5(os.path.join(inputPath, filename), "/process/acquisition/rotation/range")[0]
    #         _, _, _, anglelist = dxchange.exchange.read_aps_tomoscan_hdf5(os.path.join(inputPath, filename))

print(f'{filename}: \nslices: {numslices}, rays: {numrays}, angles: {numangles}, angularrange: {angularrange}, \npxsize: {pxsize*10000} um, distance: {propagation_dist} mm. energy: {kev} keV')
if kev>100:
    print('white light mode detected; energy is set to 30 kev for the phase retrieval function')

In [ ]:
#Edit parameters for recon. Open the reconstruction.py file to see comments on options.
functioninput = {
    "filename": filename, "inputPath": inputPath, "outputPath": outputPath, "filetype": filetype, "timepoint": 0,
    "sinoused": (-1,1,1), #(start, stop, step) for which slices to recon. (-1,1,1) reconstructs just the middle slice
    "cor": None, #None, # None, #set something other than None if you want to automatically detect center of rotation
    "corFunction": 'pc',
    "writereconstruction": 0, "writenormalized": 0, "dorecon": 1,
    "butterworth_cutoff": 0.25,
    "doOutliers1D": 1,
    "doFWringremoval": 1,
    "doPhaseRetrieval": 0, "alphaReg": 0.005,
    "castTo8bit": 0, "cast8bit_min": -1, "cast8bit_max": 1,
    "doPolarRing": 0, "Rarc": 15, "Rmaxwidth": 15, "Rtmax": 5, "Rthr": 5, "Rtmin": -2,
    "chunk_proj": 100, "chunk_sino": 25,
    "verbose_printing": 1,
    "angle_offset": 0,
    "use360to180": 0,
}
recon_dictionary, cor_tomo = recon_setup(**functioninput)
print(f"center of rotation: {recon_dictionary['cor']}")

In [ ]:
#manually set center of rotation based on overlay of first and last image (assumes 180 degree rotation)
if isinstance(cor_tomo,np.ndarray):  
    firstimage = np.squeeze(cor_tomo[0,:,:])
    lastimageflipped = np.squeeze(cor_tomo[1,:,::-1])
    shiftedlastimage = np.zeros_like(lastimageflipped)
    imagewidth = cor_tomo.shape[2]
    cor_shift_initial_value = 2*(recon_dictionary['cor'] - imagewidth/2)
    non = lambda s: s if s<0 else None
    mom = lambda s: max(0,s)

    fig, ax = plt.subplots(constrained_layout=True)
    fig.canvas.toolbar_position = 'right'
    fig.canvas.header_visible = False
    img = ax.imshow(firstimage-lastimageflipped,cmap='gray',vmin=-.1,vmax=.1)
    
    def updateimage(ox):
        recon_dictionary['cor'] = imagewidth/2.0 + ox/2.0
        shiftedlastimage[:,mom(ox):non(ox)] = lastimageflipped[:, mom(-ox):non(-ox)]
        img.set_array(firstimage - shiftedlastimage)
        ax.set_title(f"COR: {recon_dictionary['cor']}")
        fig.canvas.draw()
        fig.canvas.flush_events()
    
    slider_xshift = widgets.IntSlider(description='Shift Image', min = -500, max = 500, step=1, value = cor_shift_initial_value, layout=widgets.Layout(width='50%'))
    uishiftsliders = widgets.HBox([slider_xshift])
    outshiftsliders = widgets.interactive_output(updateimage,{'ox':slider_xshift});
    display(uishiftsliders,outshiftsliders)
else:
    print("COR was already set by user, no display available")

In [ ]:
#this cell reconsructs one slice
#(or the number of slices specified in sinoused)
print(f"COR: {recon_dictionary['cor']}")
rec, tomo = recon(**recon_dictionary)

In [ ]:
#not sure if this makes a copy in memory? if so you can do this another way?
imagetoshow = rec
#we assume that we want to iterate over the first dimension
number_of_slices = imagetoshow.shape[0]

def showstack(slicenumber):
    plt.figure(figsize=(8, 8))
    plt.imshow(imagetoshow[slicenumber,:,:],cmap='gray',vmin=-5,vmax=30)
    return slicenumber
w = widgets.interactive(showstack, slicenumber = (0,number_of_slices-1,1))
w.children[0].value = 0    
display(w)

In [ ]:
#This cell takes the parameters most recently used and
#adds doing a full reconstruction to the to-do list
functioninput['sinoused'] = None
functioninput['writereconstruction'] = 1
functioninput['verbose_printing'] = 0
functioninput['cor'] = recon_dictionary['cor']
recon_dictionary, cor_tomo = recon_setup(**functioninput)

recons_todo.append(recon_dictionary)
with open(os.path.join(outputPath,pickledparamsfile), 'wb') as f:
    pickle.dump(recons_todo, f, protocol=pickle.HIGHEST_PROTOCOL)
print(f'Parameters added to list saved in {os.path.join(outputPath,pickledparamsfile)}')
print(f'Number of reconstructions in recons_todo = {len(recons_todo)}')

In [ ]:
for i in range(0,len(recons_todo)):
    recon_dictionary = recons_todo[i]
    rec, tomo = recon(**recon_dictionary)
    
#these lines reset the queue and make a new pkl file
#in case you want to start a new set of processing with the same subdirectory
#without overwriting the last one
recons_todo = []
pklbase, pklext = os.path.splitext(pickledparamsfile)
pickledparamsfile = pklbase+'_1'+pklext

The rest of the cells are in case you lost track of what you were doing
or had to restart the notebook and want to load a previous to do list

In [ ]:
# list the data sets on recon_todo
for i in range(0,len(recons_todo)):
    print(recons_todo[i]['filename'])

In [ ]:
#If a parameters file is saved previously but wasn't run,
#it can be opened here to run the reconstructions in the next cell

# with open(os.path.join(outputPath,pickledparamsfile), 'rb') as f:
#     recons_todo = pickle.load(f)

In [ ]:
#uncomment the next lines if you want to just loop through
#a bunch of data sets and reconstruct them without first previewing them

firstfile = 15
lastfile = 23
for i in range(firstfile,lastfile+1,1):
    filename = filenamelist[i]
    functioninput = {
        "filename": filename, "inputPath": inputPath, "outputPath": outputPath, "filetype": 'dxfile', "timepoint": 0,
        "sinoused": None, #(start, stop, step) for which slices to recon. (-1,1,1) reconstructs just the middle slice
        "cor": None, # None, #set something other than None if you want to automatically detect center of rotation
        "corFunction": 'pc',
        "writereconstruction": 1, "writenormalized": 0, "dorecon": 1,
        "doOutliers1D": 1,
        "doFWringremoval": 0,
        "doPhaseRetrieval": 0, "alphaReg": 0.005,
        "castTo8bit": 0, "cast8bit_min": -1, "cast8bit_max": 1,
        "doPolarRing": 1, "Rarc": 15, "Rmaxwidth": 15, "Rtmax": 5, "Rthr": 5, "Rtmin": -2,
        "chunk_proj": 100, "chunk_sino": 25,
        "verbose_printing": 1,
        "angle_offset":0,
    }
    recon_dictionary, cor_tomo = recon_setup(**functioninput)
    print(f"center of rotation: {recon_dictionary['cor']}")
    rec, tomo = recon(**recon_dictionary)

In [ ]:
#to write out the raw data as .tif files
filename = filenamelist[615]
outputFilename = os.path.splitext(filename)[0]
fulloutputPath = os.path.join(outputPath, 'norm' + outputFilename)
if not os.path.exists(fulloutputPath):
    os.mkdir(fulloutputPath)
filenametowrite = os.path.join(fulloutputPath,outputFilename)

tomo, flat, dark, coranglelist = dxchange.exchange.read_aps_tomoscan_hdf5(os.path.join(inputPath, filename))

# dxchange.write_tiff_stack(tomo, fname=filenametowrite+'raw',start=0)
# dxchange.write_tiff_stack(flat, fname=filenametowrite+'flat',start=0)
# dxchange.write_tiff_stack(dark, fname=filenametowrite+'dark',start=0)


import tomopy

tomo = tomo.astype(np.float32)
tomopy.normalize(tomo, flat, dark, out=tomo)

dxchange.write_tiff_stack(tomo, fname=filenametowrite+'norm')
# tomomedian = np.median(tomo,axis=0);
# dxchange.write_tiff(tomomedian, fname=filenametowrite+'norm')

In [ ]:
# return a set of reconstructed slices as an array, each slice from a different value of a parameter you iterate over
keytotest = 'cor'
variable_radius = 6
variable_steps = 5
variable_center = recon_dictionary[keytotest] #use recon_dictionary[keytotest] to start from the value set before, or choose another value

if 1:
    arrayofvalues = np.linspace(variable_center-variable_radius, variable_center+variable_radius, num=variable_steps)
    rec_variations = np.zeros([len(arrayofvalues),int(recon_dictionary['numrays']),int(recon_dictionary['numrays'])])
    for i in range(0,len(arrayofvalues)):
        print(f'Running recon {i+1} of {len(arrayofvalues)}, with {keytotest}={arrayofvalues[i]}')
        recon_dictionary[keytotest] = arrayofvalues[i]
        rec, tomo = recon(**recon_dictionary)
        rec_variations[i,:,:] = rec[int(np.floor(rec.shape[0]/2)),:,:]

In [ ]:
# # return a set of reconstructed slices as an array, each slice from a different value of a parameter you iterate over
# recon_dictionary['doPhaseRetrieval'] = 1
# if ('phase_retrieval' not in recon_dictionary['function_list']):
#     recon_dictionary['function_list'].insert(2,'phase_retrieval')

# arrayofvalues = [0.001,0.0001,0.00001,0.000001,0.0000001, 0.00000001,0.000000001,.0000000001]
# keytotest = 'alphaReg'

# rec_variations = np.zeros([len(arrayofvalues),int(recon_dictionary['numrays']),int(recon_dictionary['numrays'])])
# for i in range(0,len(arrayofvalues)):
#     print(f'Running recon {i+1} of {len(arrayofvalues)}, with {keytotest}={arrayofvalues[i]}')
#     recon_dictionary[keytotest] = arrayofvalues[i]
#     rec, tomo = recon(**recon_dictionary)
#     rec_variations[i,:,:] = rec[int(np.floor(rec.shape[0]/2)),:,:]

In [ ]:
#not sure if this makes a copy in memory? if so you can do this another way?
imagetoshow = rec_variations
#we assume that we want to iterate over the first dimension
number_of_slices = imagetoshow.shape[0]

def showstack(slicenumber):
    plt.figure(figsize=(8, 8))
    plt.imshow(imagetoshow[slicenumber,:,:],cmap='gray',vmin=-.1,vmax=10)
    return slicenumber
w = widgets.interactive(showstack, slicenumber = (0,number_of_slices-1,1))
w.children[0].value = 0    
display(w)